# Finance Metrics Demo 
This notebook demonstrates how to fetch financial and economic metrics
for retail demand forecasting.

In [ ]:
%autoreload 2
%reload_ext autoreload

import pandas as pd
import numpy as np
import sys

# Add src to path to import our package
sys.path.insert(0, '../src')

from finance_metrics import MarketIndices, CommodityPrices

## 1. Market Indices

Market indices reflect economic sentiment and wealth effects that influence
consumer spending behavior.

In [ ]:
market = MarketIndices()

# Fetch S&P 500 data for the last year
sp500 = market.get_index("sp500", start_date="2023-01-01", end_date="2024-01-01")
print("S&P 500 Data:")
print(sp500.head())
print(f"\nShape: {sp500.shape}")

## 2. Data Aggregation for Weekly Forecasting

Example of how to aggregate daily data to weekly intervals for alignment with your forecasting model.

In [ ]:
# Aggregate daily market data to weekly
daily_sp500 = market.get_index("sp500", start_date="2023-01-01", end_date="2024-01-01")

# Resample to weekly frequency (Sunday as end of week)
# Apply aggregation to entire DataFrame, not just 'Close' series
weekly_agg = daily_sp500.resample('W').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum'
})

print("Weekly Aggregated Data:")
display(weekly_agg.head(10))

# Calculate weekly features for forecasting
weekly_features = pd.DataFrame({
    'close': weekly_agg['Close'],
    'weekly_return': weekly_agg['Close'].pct_change(),
    'volatility': daily_sp500['Close'].resample('W').std(),
    'avg_volume': daily_sp500['Volume'].resample('W').mean()
})

print("\nWeekly Features for Forecasting:")
display(weekly_features.head())

## 2. Commodity Prices

Commodity prices (especially energy) affect consumer costs and purchasing power.

In [ ]:
commodities = CommodityPrices()

# Get oil spread (WTI vs Brent)
oil_spread = commodities.get_oil_spread(start_date="2023-01-01", end_date="2024-01-01")
print("\nOil Spread (Brent - WTI):")
display(oil_spread.head())
print(f"\nAverage Spread: ${oil_spread['spread'].mean():.2f}")

# Fetch WTI crude oil prices
oil = commodities.get_commodity("crude_oil_wti", start_date="2023-01-01", end_date="2024-01-01")
print("WTI Crude Oil Prices:")
display(oil.head())
print(f"\nAverage Price: ${oil['Close'].mean():.2f}")
print(f"Price Range: ${oil['Close'].min():.2f} - ${oil['Close'].max():.2f}")

In [ ]:
# Calculate weekly returns (aligned with weekly forecasting)
weekly_sp500 = market.get_index("sp500", start_date="2023-01-01", interval="1wk")
weekly_returns = weekly_sp500['Close'].pct_change()
print("\nWeekly Returns (for alignment with weekly forecasts):")
print(weekly_returns.head(10))
print(f"\nMean Weekly Return: {weekly_returns.mean():.4f}")
print(f"Std Dev: {weekly_returns.std():.4f}")

In [ ]:
# Fetch multiple indices at once for comparison
indices = market.get_multiple_indices(
    ["sp500", "nasdaq", "vix"],
    start_date="2023-01-01",
    end_date="2024-01-01"
)
print("\nMultiple Indices:")
print(indices.head())
print(f"\nCorrelation Matrix:")
print(indices.corr())

## Next Steps

1. **Implement FRED API integration** for macroeconomic indicators (GDP, CPI, unemployment)
2. **Add consumer metrics** (consumer confidence, retail sales)
3. **Create feature engineering utilities** (lagged features, rolling windows, seasonal decomposition)
4. **Build correlation analysis** between economic metrics and retail demand
5. **Add data quality checks** and outlier detection

For more information, see the README.md file.